In [1]:
import pandas as pd
import numpy as np
import replicaEVSE.load_curve as sim
import replicaEVSE.datautils as simdu
import os
import joblib
import dask.dataframe as dd

%reload_ext autoreload
%autoreload 2

pd.set_option('display.max_columns', None)

datadir = '../../data/'
mode = 'PRIVATE_AUTO'
test = False

In [2]:


#Created in the EIA_data_download.ipynb notebook
existing_load=pd.read_csv(datadir+'EIA_demand_summary.csv') 
if test:
    # df = pd.read_parquet(os.path.join(datadir, 'wa_pop_and_trips_subsample.parquet'))
    df = pd.read_parquet(os.path.join(datadir, 'wa_pop_and_trips_sorted_county.parquet'))
    df = df.head(10000)
    df = df.loc[df['mode'] == mode]
    simulation_id = 'dev'

else: 
    # read in the joined trips and population data sets
    merged_df = pd.read_parquet(os.path.join(datadir, 'wa_pop_and_trips_sorted_county.parquet'))

    # right now, only look at private auto trips
    df = merged_df.loc[merged_df['mode'] == mode]
    # take out the mobile and commercial MHDV

### TODO: revisit taking out mobile home owners
df = df[(df['building_type'] != 'mobile') & (df['building_type'] != None)]



###  sample by county: for now use a percent until we have the stock rollover model

`ev_df` should be given to us to run. 

In [4]:
stock_rollover = pd.read_csv(datadir+'LDV_pop_adjusted.csv')
efficiency = pd.read_csv(datadir+'vehicle_inputs.csv')

In [5]:
personal = ['Personal Sedan', 'Personal Crossover', 'Personal Truck/SUV']
commercial = ['Commercial Sedan', 'Commercial Crossover', 'Commercial Truck/SUV']
for cartype in personal + commercial:
    for powertrain in  ['EV', 'PHEV']:
        cond = (efficiency['Vehicle_type']==cartype) & ~efficiency['Powertrain'].isin(['ICE-G', 'ICE-D', 'FCEV']) & (efficiency['Powertrain']==powertrain) & (efficiency['Vintage'] == 2022)
        ef = efficiency[cond]['Efficiency'].values[0]
        print(cartype, powertrain, ef)

Personal Sedan EV 0.25
Personal Sedan PHEV 80.0
Personal Crossover EV 0.3
Personal Crossover PHEV 80.0
Personal Truck/SUV EV 0.49
Personal Truck/SUV PHEV 80.0
Commercial Sedan EV 0.25
Commercial Sedan PHEV 80.0
Commercial Crossover EV 0.3
Commercial Crossover PHEV 80.0
Commercial Truck/SUV EV 0.49
Commercial Truck/SUV PHEV 80.0


# extract the population ldv data from the stock rollover model 
 - What to do about housing == 'other'? 

### include both EV and PHEV

In [9]:
ev_cond = stock_rollover['Powertrain'].isin(['EV', 'PHEV'])
# ev_cond = (stock_rollover['Powertrain']=='EV') | (stock_rollover['Powertrain']=='PHEV')
# nev_df = stock_rollover[(stock_rollover['Powertrain']=='EV')].copy()
nev_df = stock_rollover[ev_cond].copy()
nev_df = nev_df[nev_df['domicile'] != 'other'].copy()
print(len(nev_df))


924


See the change in stock over the years

In [10]:
nev_df[['2022', '2035']]

,2022,2035
0,0,1
3,0,0
5,0,5
8,0,1
9,0,0
...,...,...
2377,98,14941
2378,34,22073
2381,18,4426
2385,133,82306


We ingest the number of vehicles in each segment for each year and sample from that subset of the trips table for each county segment and year. This takes > 300 minutes for all the years and segments

In [8]:
personal = ['Personal Sedan', 'Personal Crossover', 'Personal Truck/SUV']
commercial = ['Commercial Sedan', 'Commercial Crossover', 'Commercial Truck/SUV']

if True:
    for year in [2023, 2025, 2030, 2035]:
        #segment_persons = []
        for segment in personal+commercial:
            seg_string = segment.replace(" ", "_").lower().replace("/suv", "")
            simulation_id = f'{seg_string}_{str(year)}'
            print(simulation_id)
            if "Sedan" in segment:
                eff = 0.25
            elif "Crossover" in segment:
                eff = 0.30
            elif 'Truck' in segment:
                eff = 0.49
            else:
                print('no match')
                eff = 0.3
            print(eff)
            nev_df = stock_rollover[stock_rollover['Powertrain']=='EV'].copy()
            nev_df = nev_df[nev_df['Vehicle_type']==segment].copy()
            nev_df = nev_df[nev_df['domicile'] != 'other'].copy()
            
            # we dont want to include the same people in multiple segments.
            # this results in not enough people in some small counties...
            #if len(segment_persons) == 0:
            #    df_county_subset = simdu.sample_people_by_county(df, nev_df, year=year) 
            #else:
            #    df = df[~df['person_id'].isin(segment_persons)]    
            # ~3 mins
            df_county_subset = simdu.sample_people_by_county(df, nev_df, year=year)
            df_county_subset['efficiency'] = eff
            # segment_persons.extend(df_county_subset['person_id'].tolist())
            # ddf = dd.from_pandas(df_county_subset, npartitions=500)
            df_county_subset.to_parquet(os.path.join(datadir, f'county_samples/county_sample_{simulation_id}.parquet'), 
                                        engine='pyarrow',
                                        compression='snappy')

personal_sedan_2023
0.25
81 Adams mfh
346 Adams sfh
53 Asotin mfh
253 Asotin sfh
586 Benton mfh
1773 Benton sfh
313 Chelan mfh
878 Chelan sfh
198 Clallam mfh
1062 Clallam sfh
1507 Clark mfh
4783 Clark sfh
7 Columbia mfh
56 Columbia sfh
301 Cowlitz mfh
1071 Cowlitz sfh
108 Douglas mfh
505 Douglas sfh
10 Ferry mfh
100 Ferry sfh
262 Franklin mfh
841 Franklin sfh
3 Garfield mfh
38 Garfield sfh
315 Grant mfh
1069 Grant sfh
249 Grays Harbor mfh
771 Grays Harbor sfh
264 Island mfh
1198 Island sfh
115 Jefferson mfh
709 Jefferson sfh
15587 King mfh
23389 King sfh
1001 Kitsap mfh
3026 Kitsap sfh
261 Kittitas mfh
513 Kittitas sfh
59 Klickitat mfh
359 Klickitat sfh
190 Lewis mfh
862 Lewis sfh
11 Lincoln mfh
152 Lincoln sfh
98 Mason mfh
926 Mason sfh
124 Okanogan mfh
535 Okanogan sfh
40 Pacific mfh
278 Pacific sfh
19 Pend Oreille mfh
181 Pend Oreille sfh
2802 Pierce mfh
7203 Pierce sfh
59 San Juan mfh
492 San Juan sfh
386 Skagit mfh
1534 Skagit sfh
19 Skamania mfh
178 Skamania sfh
3502 Snohomish mf

KeyboardInterrupt: 

# HERE: main debugging for the sampling

- previously each year was the same size

In [13]:
nev_df

,County,Vehicle_type,domicile,Powertrain,2022,2023,2024,2025,2026,2027,2028,2029,2030,2031,2032,2033,2034,2035,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33
0,Adams,Commercial Crossover,mfh,EV,0,0,0,0,0,0,1,1,1,1,1,1,1,1,NaN,2022.0,2023.0,2024.0,2025.0,2026.0,2027.0,2028.0,2029.0,2030.0,2031.0,2032.0,2033.0,2034.0,2035.0,NaN
3,Adams,Commercial Crossover,mfh,PHEV,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Adams,Commercial Crossover,sfh,EV,0,0,1,1,1,2,2,3,3,3,3,5,5,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Adams,Commercial Crossover,sfh,PHEV,0,1,1,1,1,1,1,1,1,1,1,1,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Adams,Commercial Sedan,mfh,EV,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2377,Yakima,Personal Sedan,sfh,PHEV,98,2366,4663,6234,7713,8798,9690,10397,11368,12060,12701,13484,14215,14941,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2378,Yakima,Personal Truck/SUV,mfh,EV,34,1803,3576,5274,6936,8566,10112,11392,12910,14537,16245,17965,19822,22073,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2381,Yakima,Personal Truck/SUV,mfh,PHEV,18,855,1410,1814,2098,2370,2564,2759,2986,3239,3499,3776,4065,4426,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2385,Yakima,Personal Truck/SUV,sfh,EV,133,6413,12701,18729,24732,30762,36553,41370,47081,53208,59661,66344,73561,82306,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
def segment_efficiency(segment):
    """This simply returns the expected efficiency (kwh/mi) for a given segment.
    It is quite simple now but could be expanded to include more complex
    efficiency calculations over the years.

    Args:
        segment (str): The segment of the vehicle

    Returns:
        float: efficiency in kWh/mile
    """

    # get the efficiency for the segment
    if "Sedan" in segment:
        eff = 0.25
    elif "Crossover" in segment:
        eff = 0.30
    elif 'Truck' in segment:
        eff = 0.49
    else:
        print('no match')
    return eff

In [14]:

for year in [2023]:
    segment_persons = []

    # we loop over segments in order to assign one car type per person.
    for segment in ['Commercial Crossover', 'Commercial Truck/SUV']:
        
        # convert to readable string for simulation id
        seg_string = segment.replace(" ", "_").lower().replace("/suv", "")
        simulation_id = f'{seg_string}_{str(year)}'
        print('+------------------------------------+')
        print(f"Current segment and year: {simulation_id}")
        print('+------------------------------------+')

        # get the efficiency for the segment
        eff = segment_efficiency(segment)

        # this is defined above and is a cut from the stock rollover raw data
        # it is the number of vehicles of a given seg per year. 
        segment_condition = (nev_df['Vehicle_type']==segment) & (nev_df['year']==str(year))
        nev_df_sub = nev_df[segment_condition].copy()
        
        # we dont want to include the same people in multiple segments.
        # this results in not enough people in some small counties...
        #if len(segment_persons) == 0:
        #    df_county_subset = simdu.sample_people_by_county(df, nev_df, year=year) 
        #else:
        #    df = df[~df['person_id'].isin(segment_persons)]    
        # ~3 mins
        df_county_subset = simdu.sample_people_by_county(df, nev_df_sub, year=year)
        df_county_subset['efficiency'] = eff
        df_county_subset['segment'] = seg_string
        # segment_persons.extend(df_county_subset['person_id'].tolist())
        # ddf = dd.from_pandas(df_county_subset, npartitions=500)
        df_county_subset.to_parquet(os.path.join(datadir, f'county_samples/county_sample_{simulation_id}.parquet'), 
                                    engine='pyarrow',
                                    compression='snappy')
        segment_persons.append(df_county_subset['person_id'].tolist())

+------------------------------------+
Current segment and year: commercial_crossover_2023
+------------------------------------+


KeyError: 'year'

In [ ]:
unique_df = df.drop_duplicates(subset=['person_id'])[['person_id', 'destination_county', 'building_type']]

In [11]:
year

'2025'

In [10]:

year = str(year)
reduced_df = []
for _, cnty in nev_df.iterrows():
    county = cnty['County']
    num_to_select = cnty[year]
    domicile = cnty['domicile']
    print(num_to_select, county, domicile)
    
    # slice the unique dataframe to only include the county
    county_df = unique_df[(unique_df['destination_county'].str.contains(county))]

    # make sure we don't select more people than are in the county
    #if num_to_select > len(county_df):
    #    num_to_select = len(county_df)
    # print(f'Warning: {num_to_select} people selected for {county} but only {len(county_df)} people in {county}')


    if domicile == 'sfh':
        county_df_sub = county_df[county_df['building_type'] == 'single_family']
    elif domicile == 'mfh':
        county_df_sub = county_df[county_df['building_type'] != 'single_family']
    else:
        print("Warning: domicile not recognized. Investigate the input df.")
    # unique people in that county

    
    selected = county_df_sub.person_id.sample(n=num_to_select, replace=False, random_state=42)

    
    # grab only those selected people from the original dataframe
    cnty_df = df[(df['person_id'].isin(selected))]
    
    print(num_to_select, len(county_df_sub), len(selected), len(cnty_df))

    # append it to the reduced dataframe
    reduced_df.append(cnty_df)

final_df = pd.concat(reduced_df)

424 Adams mfh
424 1575 424 2632
1709 Adams sfh
1709 7908 1709 10025
332 Asotin mfh
332 3974 332 980
1591 Asotin sfh
1591 22209 1591 5101
1608 Benton mfh
1608 25143 1608 9746
5379 Benton sfh
5379 108969 5379 32858
833 Chelan mfh
833 10113 833 4729
2639 Chelan sfh
2639 43624 2639 15593
528 Clallam mfh
528 6383 528 2856
3109 Clallam sfh
3109 44321 3109 16747
3476 Clark mfh
3476 72174 3476 13768
12160 Clark sfh
12160 302134 12160 52095
47 Columbia mfh
47 269 47 262
297 Columbia sfh
297 1909 297 1456
903 Cowlitz mfh
903 12285 903 4455
3578 Cowlitz sfh
3578 68261 3578 17427
332 Douglas mfh
332 3858 332 1993
1689 Douglas sfh
1689 18570 1689 9681
192 Ferry mfh
192 327 192 856
1417 Ferry sfh
1417 3097 1417 6505
713 Franklin mfh
713 10315 713 4112
2530 Franklin sfh
2530 43417 2530 14747
38 Garfield mfh
38 100 38 140
396 Garfield sfh
396 966 396 1638
951 Grant mfh
951 9144 951 5523
3553 Grant sfh
3553 39197 3553 22253
820 Grays Harbor mfh
820 8099 820 4060
2747 Grays Harbor sfh
2747 38014 2747 14

In [14]:
cnty_df['destination_county'].value_counts()

destination_county
Yakima County, WA          112371
Benton County, WA            1599
Kittitas County, WA           512
Grant County, WA              230
Klickitat County, WA          206
Franklin County, WA           205
King County, WA               182
Pierce County, WA              74
Adams County, WA               71
Spokane County, WA             64
Chelan County, WA              57
Snohomish County, WA           51
Lewis County, WA               46
Walla Walla County, WA         35
Clark County, WA               35
Douglas County, WA             29
Grays Harbor County, WA        16
Kitsap County, WA              14
Whatcom County, WA             12
Thurston County, WA            11
Whitman County, WA             11
Skagit County, WA              10
Okanogan County, WA             7
Columbia County, WA             5
Cowlitz County, WA              5
Clallam County, WA              4
Lincoln County, WA              4
Skamania County, WA             4
Stevens County, WA           

In [15]:
24725/455888 

0.05423481205910224

In [7]:
year = 2023
segment = 'Personal Sedan'
seg_string = segment.replace(" ", "_").lower().replace("/suv", "")
simulation_id = f'{seg_string}_{str(year)}'
df_county_subset = pd.read_parquet(os.path.join(datadir, f'county_sample_{simulation_id}.parquet'))

In [26]:
personal = ['Personal Sedan', 'Personal Crossover', 'Personal Truck/SUV']
commercial = ['Commercial Sedan', 'Commercial Crossover', 'Commercial Truck/SUV']

for year in np.arange(2022, 2036, 1):
    segment_persons = []
    for segment in personal+commercial:
        seg_string = segment.replace(" ", "_").lower().replace("/suv", "")
        simulation_id = f'{seg_string}_{str(year)}'
        print(simulation_id)
        if "Sedan" in segment:
            eff = 0.25
        elif "Crossover" in segment:
            eff = 0.30
        elif 'Truck' in segment:
            eff = 0.49
        else:
            print('no match')
            eff = 0.3
        print(eff)
        nev_df = stock_rollover[stock_rollover['Powertrain']=='EV'].copy()
        nev_df_seg = nev_df[nev_df['Vehicle_type']==segment].copy()
        
        
        # we dont want to include the same people in multiple segments.
        # this results in not enough people in some small counties...
        #if len(segment_persons) == 0:
        #    df_county_subset = simdu.sample_people_by_county(df, nev_df, year=year) 
        #else:
        #    df = df[~df['person_id'].isin(segment_persons)]    
        # ~3 mins
        df_county_subset = simdu.sample_people_by_county(df, nev_df, year=year)
        df_county_subset['efficiency'] = eff
        segment_persons.extend(df_county_subset['person_id'].tolist())
        # ddf = dd.from_pandas(df_county_subset, npartitions=500)
        df_county_subset.to_parquet(os.path.join(datadir, f'county_sample_{simulation_id}.parquet'), 
                                    engine='pyarrow',
                                    compression='snappy')

personal_sedan_2022
0.25
0 Adams mfh
12 Adams sfh
1 Asotin mfh
18 Asotin sfh
147 Benton mfh
458 Benton sfh
88 Chelan mfh
238 Chelan sfh
51 Clallam mfh
288 Clallam sfh
709 Clark mfh
2253 Clark sfh
0 Columbia mfh
6 Columbia sfh
55 Cowlitz mfh
198 Cowlitz sfh
16 Douglas mfh
83 Douglas sfh
0 Ferry mfh
15 Ferry sfh
37 Franklin mfh
128 Franklin sfh
0 Garfield mfh
1 Garfield sfh
27 Grant mfh
103 Grant sfh
35 Grays Harbor mfh
123 Grays Harbor sfh
101 Island mfh
457 Island sfh
47 Jefferson mfh
292 Jefferson sfh
11350 King mfh
17036 King sfh
450 Kitsap mfh
1372 Kitsap sfh
53 Kittitas mfh
103 Kittitas sfh
11 Klickitat mfh
61 Klickitat sfh
31 Lewis mfh
141 Lewis sfh
0 Lincoln mfh
5 Lincoln sfh
21 Mason mfh
228 Mason sfh
8 Okanogan mfh
46 Okanogan sfh
4 Pacific mfh
34 Pacific sfh
0 Pend Oreille mfh
15 Pend Oreille sfh
1076 Pierce mfh
2767 Pierce sfh
40 San Juan mfh
341 San Juan sfh
109 Skagit mfh
427 Skagit sfh
3 Skamania mfh
40 Skamania sfh
1764 Snohomish mfh
4312 Snohomish sfh
340 Spokane mfh
795

In [21]:

# get the unique people in the dataframe
unique_df = df.drop_duplicates(subset=['person_id'])[['person_id', 'destination_county', 'building_type']]
year = str(year)
year = '2035'
reduced_df = []
for _, cnty in nev_df.iterrows():
    county = cnty['County']
    num_to_select = cnty[year]
    domicile = cnty['domicile']
    print(num_to_select, county, domicile)

    # slice the unique dataframe to only include the county
    county_df = unique_df[(unique_df['destination_county'].str.contains(county))]

1 Adams mfh
5 Adams sfh
0 Adams mfh
13 Adams sfh
77 Adams mfh
342 Adams sfh
26 Adams mfh
122 Adams sfh
472 Adams mfh
1930 Adams sfh
1701 Adams mfh
6990 Adams sfh
0 Asotin mfh
1 Asotin sfh
0 Asotin mfh
10 Asotin sfh
38 Asotin mfh
182 Asotin sfh
31 Asotin mfh
173 Asotin sfh
306 Asotin mfh
1428 Asotin sfh
1509 Asotin mfh
7351 Asotin sfh
1 Benton mfh
12 Benton sfh
30 Benton mfh
103 Benton sfh
532 Benton mfh
1751 Benton sfh
413 Benton mfh
1389 Benton sfh
5008 Benton mfh
15037 Benton sfh
14101 Benton mfh
46149 Benton sfh
1 Chelan mfh
10 Chelan sfh
14 Chelan mfh
45 Chelan sfh
324 Chelan mfh
1004 Chelan sfh
243 Chelan mfh
785 Chelan sfh
1538 Chelan mfh
4377 Chelan sfh
6956 Chelan mfh
21602 Chelan sfh
0 Clallam mfh
12 Clallam sfh
4 Clallam mfh
62 Clallam sfh
98 Clallam mfh
599 Clallam sfh
128 Clallam mfh
792 Clallam sfh
979 Clallam mfh
5134 Clallam sfh
3912 Clallam mfh
22452 Clallam sfh
6 Clark mfh
47 Clark sfh
100 Clark mfh
340 Clark sfh
1115 Clark mfh
3862 Clark sfh
1118 Clark mfh
3961 Clark 

In [23]:
len(county_df)

29